In [1]:
import os
import pickle
from dypac import Dypac
from niak_load_confounds import load_confounds #for fmriprep: from load_confounds import load_confounds
import numpy as np
import sklearn as sk
import scipy as sp
import matplotlib.pyplot as plt
import time
from nilearn import datasets
from nilearn import image
from nilearn.input_data import NiftiMasker
from nilearn import plotting
from nilearn.decomposition import DictLearning, CanICA
from scipy.stats import pearsonr

Here we will desigate the paths to the functional data and confound files.

In [2]:
path_base = '/home/lussier/Projects/data/abide/'
path_func = 'func/'
path_conf = 'func/'

We will also load an atlas for probabilistic grey matter segmentation.

In [3]:
mni = datasets.fetch_icbm152_2009()

A text file in the functional folder containing the names of the functional files in a single column is used to generate the list for the dypac model.

In [4]:
func = []
func_file = open((os.path.join(path_base,path_func, 'functional_batch.txt')),'r')
for line in func_file:
    func.append(os.path.join(path_base,path_func, (line.strip())))
func_file.close()
print (func)

['/home/lussier/Projects/data/abide/func/fmri_sub0050004_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050006_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050007_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050008_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050009_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050010_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050012_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050013_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050014_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050015_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050016_session1_run1_n.nii.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050019_session1_run1_n.nii.gz', '/home/lussier/Projects/dat

A similar text file with the names of the confounds files in the same order as the functional is used to create the confounds file list for the model.

In [5]:
conf_list = []
conf_file = open((os.path.join(path_base,path_conf, 'confounds_batch.txt')),'r')
for line in conf_file:
    conf_list.append(os.path.join(path_base,path_conf, (line.strip())))
conf_file.close()
print (conf_list)

['/home/lussier/Projects/data/abide/func/fmri_sub0050004_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050006_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050007_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050008_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050009_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050010_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050012_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050013_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050014_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050015_session1_run1_n_confounds.tsv.gz', '/home/lussier/Projects/data/abide/func/fmri_sub0050016_session1_run1_n_confounds.tsv.gz'

We now load the confounds from the list. 

In [ ]:
conf = []
for idx in conf_list:
    conf.append(load_confounds(idx))
conf

[     motion_pca_1  motion_pca_2  motion_pca_3  motion_pca_4  slow_drift  \
 0       -0.697245      0.187010     -0.234231     -0.101055    1.403441   
 1       -0.694118      0.217728     -0.213696     -0.087715    1.403260   
 2       -0.682421      0.203183     -0.183595     -0.062993    1.402716   
 3       -0.660903      0.185995     -0.173975     -0.096137    1.401810   
 4       -0.621409      0.198322     -0.182370      0.053088    1.400542   
 ..            ...           ...           ...           ...         ...   
 191      0.880470     -0.552727     -0.152051     -0.090662   -1.413307   
 192      1.247066     -0.294551      0.078100      0.213544   -1.414937   
 193      1.086774     -0.189042     -0.150654     -0.070050   -1.416205   
 194      0.998692     -0.038633     -0.140128     -0.083217   -1.417110   
 195      0.901744      0.333655     -0.125369     -0.177660   -1.417654   
 
      slow_drift.1  slow_drift.2  slow_drift.3  slow_drift.4    wm_avg  \
 0        1.

Now we run dypac. If you already have a saved model that you would like to load instead, skip down. First we specify the model, with the main relevant options. Then we fit the model on the functional data.

In [ ]:
model = Dypac(subsample_size=30, n_clusters=20, grey_matter=mni.gm, n_states=500, 
              verbose=1, n_init=1, n_init_aggregation=1, n_replications=20, detrend=True, 
              smoothing_fwhm=5, standardize=True, n_batch=4, threshold_sim=0.2)
model.fit(func, confounds=conf)

[MultiNiftiMasker.fit] Loading data from [/home/lussier/Projects/data/abide/func/fmri_sub0050004_session1_run1_n.nii.gz, /home/lussier/Projects/data/abide/func/fmri_sub0050006_session1_run1_n.nii.gz, /home/lussier/Projects/data/abide/func/fm
[MultiNiftiMasker.fit] Computing mask
[MultiNiftiMasker.transform] Resampling mask
[Dypac] Restrict brain mask to grey matter
[MultiNiftiMasker.fit] Loading data from None
[MultiNiftiMasker.transform] Resampling mask
[Dypac] Processing batch 0
[MultiNiftiMasker.transform_single_imgs] Loading data from Nifti1Image('/home/lussier/Projects/data/abide/func/fmri_sub0050004_session1_run1_n.nii.gz')
[MultiNiftiMasker.transform_single_imgs] Smoothing images
[MultiNiftiMasker.transform_single_imgs] Extracting region signals
[MultiNiftiMasker.transform_single_imgs] Cleaning extracted signals


Replicating clusters in data #0: 100%|██████████| 20/20 [00:09<00:00,  2.15it/s]


[MultiNiftiMasker.transform_single_imgs] Loading data from Nifti1Image('/home/lussier/Projects/data/abide/func/fmri_sub0050009_session1_run1_n.nii.gz')
[MultiNiftiMasker.transform_single_imgs] Smoothing images
[MultiNiftiMasker.transform_single_imgs] Extracting region signals
[MultiNiftiMasker.transform_single_imgs] Cleaning extracted signals


Replicating clusters in data #1: 100%|██████████| 20/20 [00:08<00:00,  2.35it/s]


[MultiNiftiMasker.transform_single_imgs] Loading data from Nifti1Image('/home/lussier/Projects/data/abide/func/fmri_sub0050014_session1_run1_n.nii.gz')
[MultiNiftiMasker.transform_single_imgs] Smoothing images
[MultiNiftiMasker.transform_single_imgs] Extracting region signals
[MultiNiftiMasker.transform_single_imgs] Cleaning extracted signals


...

Replicating clusters in data #86: 100%|██████████| 20/20 [00:08<00:00,  2.48it/s]


[MultiNiftiMasker.transform_single_imgs] Loading data from Nifti1Image('/home/lussier/Projects/data/abide/func/fmri_sub0050734_session1_run1_n.nii.gz')
[MultiNiftiMasker.transform_single_imgs] Smoothing images
[MultiNiftiMasker.transform_single_imgs] Extracting region signals
[MultiNiftiMasker.transform_single_imgs] Cleaning extracted signals


Replicating clusters in data #87: 100%|██████████| 20/20 [00:09<00:00,  2.13it/s]


[MultiNiftiMasker.transform_single_imgs] Loading data from Nifti1Image('/home/lussier/Projects/data/abide/func/fmri_sub0050743_session1_run1_n.nii.gz')
[MultiNiftiMasker.transform_single_imgs] Smoothing images
[MultiNiftiMasker.transform_single_imgs] Extracting region signals
[MultiNiftiMasker.transform_single_imgs] Cleaning extracted signals


Replicating clusters in data #88:   5%|▌         | 1/20 [00:00<00:07,  2.39it/s]

Save the model as a pickle file.

In [ ]:
pickle.dump(model, open("Dypac_abide_pass_sub30clu20sta500bat4smo5thr02.pickle", "wb"))

If there is a model already saved that we will be looking at instead, it can be loaded here.

In [ ]:
file = open("Dypac_abide_pass_sub30clu20sta500bat4smo5thr02.pickle", "rb")
model = pickle.load(file)

Visualize components maps.

In [ ]:
#generate an interactive map of an individual component
num_comp = 0 # the component number
comp = model.masker_.inverse_transform(model.components_[num_comp,:].todense())
plotting.view_img(comp, threshold=0.1, vmax=1, title="Dwell time: {dt}".format(dt=model.dwell_time_[num_comp]))

In [ ]:
#interate through the components and plot them
num_comp = 0
while num_comp < 499:
    comp = model.masker_.inverse_transform(model.components_[num_comp,:].todense())
    plotting.plot_stat_map(comp, threshold=0.2, vmax=1, colorbar=False, black_bg=True, title="Dwell time: {dt}".format(dt=model.dwell_time_[num_comp]))
    plotting.show()
    num_comp += 1

We can project the data of one subject in the parcellation space, and visualize the time course of a parcel. Note that we shift the component index, because the first coefficient of the parcellation space corresponds to the intercept, which is not included in `components_`.

In [ ]:
num_s = 0 # the subject number
weights = model.transform(func[num_s])
plt.plot(weights[:, num_comp + 1])

Let's now visualize one volume of the 4D data (after preprocessing is applied):

In [ ]:
num_t = 0 # the frame number
img = model.load_img(func[num_s])
plotting.view_img(image.index_img(img, [num_t]), vmax=4)

We can compress the data in the parcellation space, to look if the approximation is accurate.

In [ ]:
img_r = model.compress(img)
plotting.view_img(image.index_img(img_r, [num_t]), vmax=4)

It is possible to generate a R2 map, which quantifies the quality of this embedding. It computes the fraction of the variance of fMRI time series captured by the parcels. A score of 1 means perfect approximation. The score can be negative, in which case the parcellation approximation performs worst than the average of the signal. 

In [ ]:
score = model.score(img)
plotting.view_img(score, vmax=1)

We can save the R2 maps as nifti files for further processing or use with Nistats/Nilearn. 